In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Layer
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import mean_squared_error
from tensorflow.keras import backend as K

class RBFLayer(Layer):
    def __init__(self, units, gamma, **kwargs):
        super(RBFLayer, self).__init__(**kwargs)
        self.units = units
        self.gamma = gamma

    def build(self, input_shape):
        # Create a trainable weight variable for the centers of the RBFs
        self.centers = self.add_weight(name='centers',
                                      shape=(self.units, input_shape[-1]),
                                      initializer='uniform',
                                      trainable=True)
        super(RBFLayer, self).build(input_shape)

    def call(self, inputs):
        # Calculate the radial basis functions for each input sample and each RBF center
        diff = K.expand_dims(inputs) - self.centers  # shape: (batch_size, units, input_dim)
        norm = K.sum(K.square(diff), axis=-1)  # shape: (batch_size, units)
        rbf = K.exp(-self.gamma * norm)  # shape: (batch_size, units)

        return rbf

    def compute_output_shape(self, input_shape):
        return input_shape[0], self.units
    
url = 'https://raw.githubusercontent.com/ravsssh/HandballPrediction/main/Handball%20Prediction/datashotiqr/LW.csv'
df = pd.read_csv(url, delimiter=',')
# Menentukan input(X) dan output (y)
X = df.drop(columns=["SHOT","GOAL","7MSHOT","6MSHOT","9MSHOT","WINGSHOT","PIVOTSHOT","FASTBREAKSHOT"])
y = df.GOAL
# Membagi dataset 80:20 untuk train dan test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# normalisasi dataset agar tidak bias
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

validation_results_df = pd.DataFrame(columns=['R2_Test', 'MSE', 'MAE'])

for _ in range(20):
    model = Sequential()
    model.add(Flatten(input_shape=(6,)))
    model.add(RBFLayer(6, 0.5))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer='rmsprop', loss=mean_squared_error, metrics=['mean_absolute_error']) 
    history = model.fit(X_train, y_train, batch_size=8, epochs=100, validation_data=(X_test, y_test))
    
    pred_test = model.predict(X_test)
    
    r2_test = r2_score(y_test, pred_test)
    mse = history.history['val_loss'][-1]  # Last MSE from validation data
    mae = history.history['val_mean_absolute_error'][-1]  # Last MAE from validation data
    
    print(f"R-Squared Test Score: {r2_test}")
    
    # Append results to the validation results DataFrame
    validation_results_df = validation_results_df.append({'R2_Test': r2_test,
                                                          'MSE': mse,
                                                          'MAE': mae}, ignore_index=True)
    
    # Reset the TensorFlow session to clear the model
    tf.keras.backend.clear_session()

# Save the validation results DataFrame to a CSV file
validation_results_df.to_csv('validation_resultsacc.csv', index=False)


Epoch 1/100
233/233 [==============================] - 1s 2ms/step - loss: 7.4752 - mean_absolute_error: 1.9629 - val_loss: 7.7745 - val_mean_absolute_error: 2.0693
Epoch 2/100
233/233 [==============================] - 0s 1ms/step - loss: 6.2403 - mean_absolute_error: 1.8952 - val_loss: 6.8287 - val_mean_absolute_error: 1.9877
Epoch 3/100
233/233 [==============================] - 0s 1ms/step - loss: 5.5206 - mean_absolute_error: 1.8049 - val_loss: 6.0176 - val_mean_absolute_error: 1.8737
Epoch 4/100
233/233 [==============================] - 0s 1ms/step - loss: 4.7927 - mean_absolute_error: 1.6899 - val_loss: 5.1540 - val_mean_absolute_error: 1.7362
Epoch 5/100
233/233 [==============================] - 0s 1ms/step - loss: 4.0578 - mean_absolute_error: 1.5579 - val_loss: 4.4007 - val_mean_absolute_error: 1.6042
Epoch 6/100
233/233 [==============================] - 0s 1ms/step - loss: 3.4756 - mean_absolute_error: 1.4353 - val_loss: 3.8377 - val_mean_absolute_error: 1.4963
Epoch 7/10

C:\Users\Wandi\AppData\Local\Temp\ipykernel_135896\3134803166.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  validation_results_df = validation_results_df.append({'R2_Test': r2_test,


233/233 [==============================] - 1s 2ms/step - loss: 7.9041 - mean_absolute_error: 1.9331 - val_loss: 7.7387 - val_mean_absolute_error: 2.0417
Epoch 2/100
233/233 [==============================] - 0s 1ms/step - loss: 6.3359 - mean_absolute_error: 1.8741 - val_loss: 6.6544 - val_mean_absolute_error: 1.9483
Epoch 3/100
233/233 [==============================] - 0s 1ms/step - loss: 5.5307 - mean_absolute_error: 1.7882 - val_loss: 5.7832 - val_mean_absolute_error: 1.8182
Epoch 4/100
233/233 [==============================] - 0s 1ms/step - loss: 4.6820 - mean_absolute_error: 1.6482 - val_loss: 4.8458 - val_mean_absolute_error: 1.6703
Epoch 5/100
233/233 [==============================] - 0s 1ms/step - loss: 3.8842 - mean_absolute_error: 1.4975 - val_loss: 4.0699 - val_mean_absolute_error: 1.5395
Epoch 6/100
233/233 [==============================] - 0s 1ms/step - loss: 3.3105 - mean_absolute_error: 1.3894 - val_loss: 3.5509 - val_mean_absolute_error: 1.4175
Epoch 7/100
233/233 [=

C:\Users\Wandi\AppData\Local\Temp\ipykernel_135896\3134803166.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  validation_results_df = validation_results_df.append({'R2_Test': r2_test,


233/233 [==============================] - 1s 2ms/step - loss: 14.2130 - mean_absolute_error: 3.0444 - val_loss: 11.8140 - val_mean_absolute_error: 2.4459
Epoch 2/100
233/233 [==============================] - 0s 1ms/step - loss: 8.2628 - mean_absolute_error: 2.0273 - val_loss: 8.0987 - val_mean_absolute_error: 2.0697
Epoch 3/100
233/233 [==============================] - 0s 1ms/step - loss: 6.3864 - mean_absolute_error: 1.8999 - val_loss: 6.8871 - val_mean_absolute_error: 1.9798
Epoch 4/100
233/233 [==============================] - 0s 1ms/step - loss: 5.5333 - mean_absolute_error: 1.8092 - val_loss: 6.0854 - val_mean_absolute_error: 1.8618
Epoch 5/100
233/233 [==============================] - 0s 1ms/step - loss: 4.8631 - mean_absolute_error: 1.6887 - val_loss: 5.4380 - val_mean_absolute_error: 1.7545
Epoch 6/100
233/233 [==============================] - 0s 1ms/step - loss: 4.3371 - mean_absolute_error: 1.5932 - val_loss: 4.9534 - val_mean_absolute_error: 1.6662
Epoch 7/100
233/233 

...

C:\Users\Wandi\AppData\Local\Temp\ipykernel_135896\3134803166.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  validation_results_df = validation_results_df.append({'R2_Test': r2_test,


233/233 [==============================] - 1s 2ms/step - loss: 13.0137 - mean_absolute_error: 2.9939 - val_loss: 10.4686 - val_mean_absolute_error: 2.3613
Epoch 2/100
233/233 [==============================] - 0s 1ms/step - loss: 7.2590 - mean_absolute_error: 1.8438 - val_loss: 7.1033 - val_mean_absolute_error: 1.9042
Epoch 3/100
233/233 [==============================] - 0s 1ms/step - loss: 5.7250 - mean_absolute_error: 1.7737 - val_loss: 6.3157 - val_mean_absolute_error: 1.8799
Epoch 4/100
233/233 [==============================] - 0s 1ms/step - loss: 5.2610 - mean_absolute_error: 1.7444 - val_loss: 5.8750 - val_mean_absolute_error: 1.8325
Epoch 5/100
233/233 [==============================] - 0s 1ms/step - loss: 4.9110 - mean_absolute_error: 1.6950 - val_loss: 5.5094 - val_mean_absolute_error: 1.7767
Epoch 6/100
233/233 [==============================] - 0s 1ms/step - loss: 4.5996 - mean_absolute_error: 1.6404 - val_loss: 5.1807 - val_mean_absolute_error: 1.7142
Epoch 7/100
233/233 

C:\Users\Wandi\AppData\Local\Temp\ipykernel_135896\3134803166.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  validation_results_df = validation_results_df.append({'R2_Test': r2_test,


233/233 [==============================] - 1s 2ms/step - loss: 9.0837 - mean_absolute_error: 2.1227 - val_loss: 8.1396 - val_mean_absolute_error: 2.0756
Epoch 2/100
233/233 [==============================] - 0s 1ms/step - loss: 6.5385 - mean_absolute_error: 1.9265 - val_loss: 6.5650 - val_mean_absolute_error: 1.9772
Epoch 3/100
233/233 [==============================] - 0s 1ms/step - loss: 5.5961 - mean_absolute_error: 1.8465 - val_loss: 5.6919 - val_mean_absolute_error: 1.8555
Epoch 4/100
233/233 [==============================] - 0s 1ms/step - loss: 4.8514 - mean_absolute_error: 1.7266 - val_loss: 4.9403 - val_mean_absolute_error: 1.7259
Epoch 5/100
233/233 [==============================] - 0s 1ms/step - loss: 4.1944 - mean_absolute_error: 1.5990 - val_loss: 4.3179 - val_mean_absolute_error: 1.6123
Epoch 6/100
233/233 [==============================] - 0s 1ms/step - loss: 3.6615 - mean_absolute_error: 1.4863 - val_loss: 3.8570 - val_mean_absolute_error: 1.5169
Epoch 7/100
233/233 [=

C:\Users\Wandi\AppData\Local\Temp\ipykernel_135896\3134803166.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  validation_results_df = validation_results_df.append({'R2_Test': r2_test,
